<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/kmeans0219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

加载数据

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/kmeans0219/All_image.zip -d /content/data

Archive:  /content/drive/MyDrive/kmeans0219/All_image.zip
  inflating: /content/data/0.png     
  inflating: /content/data/1.png     
  inflating: /content/data/10.png    
  inflating: /content/data/100.png   
  inflating: /content/data/1000.png  
  inflating: /content/data/1001.png  
  inflating: /content/data/1002.png  
  inflating: /content/data/1003.png  
  inflating: /content/data/1004.png  
  inflating: /content/data/1005.png  
  inflating: /content/data/1006.png  
  inflating: /content/data/1007.png  
  inflating: /content/data/1008.png  
  inflating: /content/data/1009.png  
  inflating: /content/data/101.png   
  inflating: /content/data/1010.png  
  inflating: /content/data/1011.png  
  inflating: /content/data/1012.png  
  inflating: /content/data/1013.png  
  inflating: /content/data/1014.png  
  inflating: /content/data/1015.png  
  inflating: /content/data/1016.png  
  inflating: /content/data/1017.png  
  inflating: /content/data/1018.png  
  inflating: /content/data/101

去重

In [ ]:
import os
import torch
from torchvision import transforms
from transformers import AutoModel, AutoFeatureExtractor
from PIL import Image
import numpy as np
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# 加载ViT编码器和特征提取器
model_name = "google/vit-base-patch16-224-in21k"
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 设置图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载图像并提取特征向量
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0)  # 扩展batch维度
        with torch.no_grad():
            features = model(image).last_hidden_state
        feature_vector = features.mean(dim=1).squeeze().cpu().numpy()  # 获取特征向量的均值
        if feature_vector is not None:
            print(f"Feature extracted from {image_path}")
        return feature_vector
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# 计算余弦相似度
def compute_similarity_matrix(features_list):
    return cosine_similarity(features_list)

# 数据去重
def remove_duplicates(image_dir, output_dir, similarity_threshold=0.65):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(".jpg") or img.endswith(".png")]
    features_list = []

    # 提取图像特征
    for img_path in tqdm(image_paths, desc="Extracting features"):
        features = extract_features(img_path)
        if features is not None:
            features_list.append(features)

    # 如果没有成功提取任何特征，报错并返回空列表
    if not features_list:
        print("No features were extracted. Please check the image files.")
        return []

    # 计算余弦相似度矩阵
    similarity_matrix = compute_similarity_matrix(features_list)

    # 根据相似度阈值进行去重
    to_remove = set()
    for i in range(len(image_paths)):
        if i in to_remove:
            continue
        for j in range(i + 1, len(image_paths)):
            if similarity_matrix[i][j] > similarity_threshold:
                to_remove.add(j)

    # 保留没有被标记为删除的图像
    unique_image_paths = [image_paths[i] for i in range(len(image_paths)) if i not in to_remove]

    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 将去重后的图像复制到新的目录
    for img_path in unique_image_paths:
        shutil.copy(img_path, output_dir)

    return unique_image_paths

# 设置数据文件夹路径和输出文件夹路径
image_dir = './data'
output_dir = './data_deduplication'
unique_images = remove_duplicates(image_dir, output_dir, similarity_threshold=0.65)

# 输出去重后的图像路径
print(f"去重后的图像数量: {len(unique_images)}")
for img_path in unique_images:
    print(img_path)


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Extracting features:   0%|          | 2/1518 [00:00<02:49,  8.93it/s]

Feature extracted from ./data/1015.png
Feature extracted from ./data/169.png


Extracting features:   0%|          | 4/1518 [00:00<02:34,  9.83it/s]

Feature extracted from ./data/802.png
Feature extracted from ./data/894.png


Extracting features:   0%|          | 5/1518 [00:00<02:36,  9.65it/s]

...

Extracting features: 100%|█████████▉| 1516/1518 [02:36<00:00, 10.19it/s]

Feature extracted from ./data/1301.png
Feature extracted from ./data/1208.png
Feature extracted from ./data/1279.png


Extracting features: 100%|██████████| 1518/1518 [02:36<00:00,  9.69it/s]

Feature extracted from ./data/1347.png
Feature extracted from ./data/344.png


去重后的图像数量: 776
./data/1015.png
./data/169.png
./data/802.png
./data/894.png
./data/1025.png
./data/1053.png
./data/580.png
./data/608.png
./data/725.png
./data/1514.png
./data/1470.png
./data/1200.png
./data/1071.png
./data/394.png
./data/325.png
./data/425.png
./data/1050.png
./data/874.png
./data/1471.png
./data/897.png
./data/46.png
./data/9.png
./data/581.png
./data/427.png
./data/1257.png
./data/165.png
./data/994.png
./data/301.png
./data/857.png
./data/1103.png
./data/114.png
./data/652.png
./data/589.png
./data/1416.png
./data/1331.png
./data/698.png
./data/187.png
./data/263.png
./data/91.png
./data/947.png
./data/1311.png
./data/745.png
./data/642.png
./data/553.png
./data/1398.png
./data/470.png
./data/478.png
./data/45.png
./data/270.png
./data/775.png
./data/688.png
./data/1107.png
./data/1127.png
./data/1486.png
./data/1461.png
./data/0.png
./data/54.png
./data/709.png
./data/481.png
./data/525.png
./data/691.png
./data/1123.png
./data/1056.png
./data/1059.png
./data/208.p

In [ ]:
!rm -r /content/data_deduplication

In [ ]:
import shutil

# 定义要压缩的文件夹路径和目标ZIP文件路径
folder_path = '/content/data_deduplication'
zip_path = '/content/drive/MyDrive/kmeans0219/data_deduplication.zip'

# 压缩文件夹为ZIP文件
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/drive/MyDrive/kmeans0219/data_unique.zip'

k-means聚类

In [2]:
!unzip /content/drive/MyDrive/kmeans0219/data_deduplication.zip -d /content/data_deduplication

Archive:  /content/drive/MyDrive/kmeans0219/data_deduplication.zip
  inflating: /content/data_deduplication/1015.png  
  inflating: /content/data_deduplication/169.png  
  inflating: /content/data_deduplication/802.png  
  inflating: /content/data_deduplication/894.png  
  inflating: /content/data_deduplication/1025.png  
  inflating: /content/data_deduplication/1053.png  
  inflating: /content/data_deduplication/580.png  
  inflating: /content/data_deduplication/608.png  
  inflating: /content/data_deduplication/725.png  
  inflating: /content/data_deduplication/1514.png  
  inflating: /content/data_deduplication/1470.png  
  inflating: /content/data_deduplication/1200.png  
  inflating: /content/data_deduplication/1071.png  
  inflating: /content/data_deduplication/394.png  
  inflating: /content/data_deduplication/325.png  
  inflating: /content/data_deduplication/425.png  
  inflating: /content/data_deduplication/1050.png  
  inflating: /content/data_deduplication/874.png  
  infla

In [3]:
import os
import shutil
import torch
from torchvision import transforms
from transformers import AutoModel, AutoFeatureExtractor
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from tqdm import tqdm

# 加载ViT编码器和特征提取器
model_name = "google/vit-base-patch16-224-in21k"
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 设置图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载图像并提取特征向量
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # 扩展batch维度
    with torch.no_grad():
        features = model(image).last_hidden_state
    return features.mean(dim=1).squeeze().cpu().numpy()  # 获取特征向量的均值

# 从去重后的图像中选择152张代表性图像
def select_representative_images(image_dir, n_clusters=152):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(".jpg") or img.endswith(".png")]

    # 提取图像特征
    features_list = []
    for img_path in tqdm(image_paths, desc="Extracting features"):
        features = extract_features(img_path)
        features_list.append(features)

    # 执行k-means聚类
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(features_list)

    # 选择每个聚类中的中心点附近的代表性图像
    selected_images = []
    for cluster_idx in range(n_clusters):
        # 找到每个聚类中心的最近图像
        cluster_center = kmeans.cluster_centers_[cluster_idx]
        distances = np.linalg.norm(np.array(features_list) - cluster_center, axis=1)
        closest_idx = np.argmin(distances)
        selected_images.append(image_paths[closest_idx])

    return selected_images

# 设置去重后的图像文件夹路径
image_dir = './data_deduplication'
output_dir = './data_kmeans'

# 确保输出文件夹存在
os.makedirs(output_dir, exist_ok=True)

# 从去重后的图像中选择152张代表性图像
representative_images = select_representative_images(image_dir, n_clusters=152)

# 将选定的152张代表性图像复制到目标文件夹
for img_path in representative_images:
    # 获取文件名
    img_name = os.path.basename(img_path)
    # 构造目标路径
    target_path = os.path.join(output_dir, img_name)
    # 复制文件
    shutil.copy(img_path, target_path)

print(f"选择的152张代表性图像已保存到: {output_dir}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Extracting features: 100%|██████████| 776/776 [01:20<00:00,  9.59it/s]


选择的152张代表性图像已保存到: ./data_kmeans


In [4]:
import shutil

# 定义要压缩的文件夹路径和目标ZIP文件路径
folder_path = '/content/data_kmeans'
zip_path = '/content/drive/MyDrive/kmeans0219/data_kmeans.zip'

# 压缩文件夹为ZIP文件
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/drive/MyDrive/kmeans0219/data_kmeans.zip'